# Plotting routine for the June2 2022 Event - PSP

In [ ]:
test = 'test'

In [104]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocess as mp
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### setting the times for py3DCORE

In [105]:
t_launch = datetime.datetime(2022, 6, 2, 7, 30, tzinfo=datetime.timezone.utc)

t_s = datetime.datetime(2022, 6, 2, 11, 45, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc)

t_fit = [
        datetime.datetime(2022, 6, 2, 12, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 13, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 13, 30, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc)
     ]


In [106]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['06/02/2022, 12:00:00', '06/02/2022, 13:00:00', '06/02/2022, 13:30:00', '06/02/2022, 14:00:00']


#### loading the pickle files

In [107]:
path = 'output/psp02062022_512_final_1_heeq/'

filepath = fp.loadpickle(path, number=-1)

In [108]:
filepath[:-7]

'output/output/psp02062022_512_final_1_heeq/09'

#### extracting parameters

In [109]:
res_mineps, res_mean, res_std, ind, keys, allres = fp.get_params(filepath, give_mineps=True)

print(ind)

2023-06-07 11:01:41.657 INFO    py3dcore.fluxplot: Retrieved the following parameters for the run with minimum epsilon:
2023-06-07 11:01:41.658 INFO    py3dcore.fluxplot:  --cme_longitude 271.68
2023-06-07 11:01:41.658 INFO    py3dcore.fluxplot:  --cme_latitude 1.97
2023-06-07 11:01:41.658 INFO    py3dcore.fluxplot:  --cme_inclination 337.14
2023-06-07 11:01:41.658 INFO    py3dcore.fluxplot:  --cme_diameter_1au 0.33
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --cme_aspect_ratio 1.14
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --cme_launch_radius 6.68
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --cme_launch_velocity 515.06
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --t_factor -205.15
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --cme_expansion_rate 1.14
2023-06-07 11:01:41.659 INFO    py3dcore.fluxplot:  --magnetic_decay_rate 1.64
2023-06-07 11:01:41.660 INFO    py3dcore.fluxplot:  --magnetic_field_strength_1au 9.53
2023-06-07 11:01:41.660 INFO   

Results :

	cme_longitude: 		268.08 +/- 6.41
	cme_latitude: 		3.04 +/- 4.72
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	t_factor: 		-251.37 +/- 29.50
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.19 +/- 1.18
	background_drag: 		1.59 +/- 0.74
	background_velocity: 		472.32 +/- 140.19
243


In [110]:
out = fp.get_ensemble_stats(filepath)
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,expansion rate,B decay rate,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.00000,512.000000,512.000000,512.000000
mean,268.077209,3.036103,338.649109,0.310422,1.382161,7.672457,447.554474,-251.369080,1.140000,1.64000,9.188761,1.591182,472.316650
std,6.416155,4.728174,5.241433,0.027132,0.284424,1.052523,102.106354,29.528316,0.000008,0.00001,1.179551,0.735838,140.326538
min,250.833237,-9.656710,323.127411,0.197226,1.002677,6.003181,200.602295,-299.853729,1.140000,1.64000,6.618775,0.206634,104.235855
25%,263.576538,-0.275850,335.175003,0.294829,1.162043,6.821921,379.335457,-275.117256,1.140000,1.64000,8.304861,0.988938,377.078690
50%,267.902603,2.910359,338.467300,0.314854,1.324374,7.535860,448.420990,-253.626732,1.140000,1.64000,9.060292,1.589923,484.754532
75%,272.458618,6.428007,342.154541,0.331970,1.536322,8.402586,515.173218,-231.726360,1.140000,1.64000,9.922786,2.181257,593.819427
max,286.319885,16.778481,353.990509,0.349836,2.535966,10.814631,743.805786,-164.543320,1.140000,1.64000,14.039117,2.994917,698.263855


In [92]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [93]:
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    f.write('Summary of results')
    f.write('\n\n')
    f.write('Results:')
    f.write('\n')
    for i in range(1, len(keys)):    
        f.write("\t{}: \t\t{:.02f} +/- {:.02f}".format(keys[i], res_mean[i], res_std[i]))
        f.write('\n')
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for i in range(1, len(keys)):
        f.write(" --{} {:.2f}".format(keys[i], res_mineps[i]))
        f.write('\n')
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')

#### loading and plotting the fitting results

In [101]:
t_s = datetime.datetime(2022, 6, 2, 10, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 6, 2, 18, tzinfo=datetime.timezone.utc)

In [103]:
fp.fullinsitu(observer='PSP', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='SPP_RTN', best=False, 
              ensemble=True, save_fig=True, legend=True, max_index=512, title=False, fit_points=True)

2023-06-07 10:46:25.957 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


2022-06-02 10:00:00+00:00 2022-06-02 18:00:00+00:00


In [95]:
fp.scatterparams(filepath)

Results :

	cme_longitude: 		-28.74 +/- 6.76
	cme_latitude: 		6.09 +/- 5.22
	cme_inclination: 		337.09 +/- 6.37
	cme_diameter_1au: 		0.32 +/- 0.02
	cme_aspect_ratio: 		1.33 +/- 0.24
	cme_launch_radius: 		10.33 +/- 1.32
	cme_launch_velocity: 		365.66 +/- 115.88
	t_factor: 		-259.38 +/- 26.80
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.84 +/- 1.34
	background_drag: 		1.60 +/- 0.76
	background_velocity: 		300.63 +/- 126.60


In [96]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

-28.741701
6.7649383544921875
1.32400381565094
115.99677276611328


In [37]:
print(res_mineps[1])

271.67804


#### Plotting the fluxrope and spacecraft positions

In [120]:
#measurement times 
#tm0 =  datetime.datetime(2022, 6, 2, 6)
tm1 =  t_launch + datetime.timedelta(hours=5)
#tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['psp'], planetlist=['Earth'], t=tm1, filepath=filepath, legend=False, view_azim=0, title=False,
                view_elev=0, view_radius=0.058, index=0)

2023-06-07 11:06:34.268 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


Results :

	cme_longitude: 		268.08 +/- 6.41
	cme_latitude: 		3.04 +/- 4.72
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	t_factor: 		-251.37 +/- 29.50
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.19 +/- 1.18
	background_drag: 		1.59 +/- 0.74
	background_velocity: 		472.32 +/- 140.19
observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x2cdf67670>


### make fluxrope 3d movie

In [83]:
def make_frame(k):
    tm0 = t_launch
    t = tm0 + k*datetime.timedelta(hours=0.5)
    
    fig = fp.full3d(spacecraftlist=['psp'], planetlist=[], legend=False, title=False,
              t=t, filepath=filepath, save_fig=True, view_radius=0.3)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/' + frmstr + '.jpg', dpi=300)  
    return fig
    

In [84]:
inn = [i for i in range(100)]


mpool = mp.Pool(processes=4)
mpool.map(make_frame, inn[0:40])
mpool.close()
mpool.join()

print('done 1/4')

mpool = mp.Pool(processes=4)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 2/4')

#mpool = mp.Pool(processes=4)
#mpool.map(make_frame, inn[40:60])
#mpool.close()
#mpool.join()

#print('done 3/4')

#mpool = mp.Pool(processes=4)
#mpool.map(make_frame, inn[60:80])
#mpool.close()
#mpool.join()

print('done 4/4')

Results :
Results :
Results :
Results :



	cme_longitude: 		268.08 +/- 6.41
	cme_longitude: 		268.08 +/- 6.41	cme_longitude: 		268.08 +/- 6.41	cme_longitude: 		268.08 +/- 6.41



	cme_latitude: 		3.04 +/- 4.72	cme_latitude: 		3.04 +/- 4.72	cme_latitude: 		3.04 +/- 4.72	cme_latitude: 		3.04 +/- 4.72


	cme_inclination: 		338.65 +/- 5.24
	cme_inclination: 		338.65 +/- 5.24	cme_inclination: 		338.65 +/- 5.24
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03

	cme_diameter_1au: 		0.31 +/- 0.03
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_aspect_ratio: 		1.38 +/- 0.28

	cme_launch_radius: 		7.67 +/- 1.05
	cme_aspect_ratio: 		1.38 +/- 0.28	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	cme_launch_radius: 		7.67 +/- 1.05


	cme_launch_velocity: 		447.55 +/- 102.01	cme_launch_velocity: 		447.55 +/- 102.01	t_f

2023-06-07 10:33:32.872 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


observer:observer:  Parker Solar ProbeParker Solar Probe

instobserver: inst  Parker Solar Probe<heliosat.spice.PSP object at 0x2fe3d5150><heliosat.spice.PSP object at 0x2fe3d5210>




2023-06-07 10:33:32.888 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data
2023-06-07 10:33:32.889 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


inst <heliosat.spice.PSP object at 0x2fe3d52d0>


...

Results :

	cme_longitude: 		268.08 +/- 6.41
	cme_latitude: 		3.04 +/- 4.72
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	t_factor: 		-251.37 +/- 29.50
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.19 +/- 1.18
	background_drag: 		1.59 +/- 0.74
	background_velocity: 		472.32 +/- 140.19
observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x310ccb2e0>


2023-06-07 10:33:49.803 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


Results :

	cme_longitude: 		268.08 +/- 6.41
	cme_latitude: 		3.04 +/- 4.72
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	t_factor: 		-251.37 +/- 29.50
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.19 +/- 1.18
	background_drag: 		1.59 +/- 0.74
	background_velocity: 		472.32 +/- 140.19
observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x3107809d0>


2023-06-07 10:33:50.081 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


done 2/4
done 4/4


In [60]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

sh: ffmpeg: command not found


32512

### plot fluxrope 3d multiview

In [31]:
fig = fp.full3d_multiview(tm1, filepath=filepath)

Results :

	cme_longitude: 		268.08 +/- 6.41
	cme_latitude: 		3.04 +/- 4.72
	cme_inclination: 		338.65 +/- 5.24
	cme_diameter_1au: 		0.31 +/- 0.03
	cme_aspect_ratio: 		1.38 +/- 0.28
	cme_launch_radius: 		7.67 +/- 1.05
	cme_launch_velocity: 		447.55 +/- 102.01
	t_factor: 		-251.37 +/- 29.50
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		9.19 +/- 1.18
	background_drag: 		1.59 +/- 0.74
	background_velocity: 		472.32 +/- 140.19
{'cme_longitude': {'maximum': 272.67803955078125, 'minimum': 270.67803955078125}, 'cme_latitude': {'maximum': 2.96509325504303, 'minimum': 0.9650932550430298}, 'cme_inclination': {'maximum': 338.1358337402344, 'minimum': 336.1358337402344}, 'cme_diameter_1au': {'maximum': 0.3417968249320984, 'minimum': 0.3217968249320984}, 'cme_aspect_ratio': {'maximum': 1.2393954753875733, 'minimum': 1.0393954753875732}, 'cme_launch_radius': {'distribution': 'uniform', 'maximum': 6.782264328002929, 'minimum': 6.58226432800

2023-06-07 09:14:27.517 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data
2023-06-07 09:14:27.541 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data
2023-06-07 09:14:27.549 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x2aeda6e30>
t_snap: 2022-06-02 12:30:00+00:00
pos [HEEQ - xyz]: [ 0.03448502 -0.06229644 -0.00434515]
observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x2aeda78e0>
t_snap: 2022-06-02 12:30:00+00:00
pos [HEEQ - xyz]: [ 0.03448502 -0.06229644 -0.00434515]
observer: Parker Solar Probe
inst <heliosat.spice.PSP object at 0x2aeda7610>
t_snap: 2022-06-02 12:30:00+00:00
pos [HEEQ - xyz]: [ 0.03448502 -0.06229644 -0.00434515]


### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

